## 20.4 LDA的变分EM算法

LDA的变分EM算法具有推理与学习效率高的优点。

### 20.4.0 KL散度

KL散度（KL divergence，Kullback-Leibler divergence）是描述<font color=blue>两个概率分布$~Q(x)~$和$~P(x)~$相似度的一种度量</font>，记作$D(Q\| P)$。

对于离散随机变量，KL散度定义为:  

$$
D(Q\|P) = \sum_{i}Q(i)\log \frac{Q(i)}{P(i)} 
$$

对于连续随机变量，KL散度定义为:

$$
D(Q\|P) = \int Q(x)\log \frac{Q(x)}{P(x)} \mathrm{d}x
$$

利用Jensen不等式很容易得到 $D(Q\|P)\geq 0$，当且仅当$Q=P$时，$D(Q\|P)= 0$  

$$
\begin{aligned} - D ( Q \| P ) & = \int Q ( x ) \log \frac { P ( x ) } { Q ( x ) } \mathrm { d } x \\ & \leqslant \log \int Q ( x ) \frac { P ( x ) } { Q ( x ) } \mathrm { d } x \\ & = \log \int P ( x ) \mathrm { d } x = 0 \end{aligned} 
$$

### 20.4.1 变分推理（variational inference）

1. MCMC通过<font color=blue>随机抽样</font>的方法近似地计算模型的后验概率；


2. VI通过解析的方法计算模型的后验概率的近似值。

**VI的基本思想：**

假设模型是联合概率分布$~p(x,z)$，其中$~x~$是观测变量（数据），$~z~$是隐变量，包括参数。  

<font color=blue>目标是学习模型的后验分布$~p(z|x)$</font>，用模型进行概率推理。

VI考虑用概率分布$~q(z)~$近似条件分布$~p(z|x)$，用KL散度 $D\left(q(z)\| p(z|x)\right)$ 计算两者的相似度，$~q(z)~$称为<font color=blue>变分分布（variational distribution）</font>

如果能找到与$~p(z|x)~$在KL散度意义下最近的分布$~q^{*}(z)~$，则可以用这个分布近似$~p(z|x)~$  

$$
p(z|x) \approx q^{*}(z)
$$

&nbsp;

$$
\begin{eqnarray}
D\left(q(z)\| p(z|x)\right) &=& \int q(z) \log \frac{q(z)}{p(z|x)} \mathrm{d}z \\
&=& \int q(z) \log q(z) \mathrm{d}z - \int q(z) \log p(z|x) \mathrm{d}z \\
&=& \int q(z) \log q(z) \mathrm{d}z - \int q(z) \log p(x,z) \mathrm{d}z + \int q(z) \log p(x) \mathrm{d}z \\
&=& E_{q}\left[\log q(z)\right] - E_{q}\left[\log p(x,z)\right] + \log p(x) \\
&=& \log p(x) - \underset{ELBO}{\underbrace{ \left\{E_{q}\left[\log p(x,z)\right] - E_{q}\left[\log q(z)\right]\right\}} } \tag{20.35} \\
&\geq& 0
\end{eqnarray}
$$

因此有  

$$
\underset{\text{evidence}}{\underbrace{\log p(x)}} \geq  \underset{\text{evidence  lower  bound, ELBO}}{\underbrace{E_{q}\left[\log p(x,z)\right] - E_{q}\left[\log q(z)\right]}} \tag{20.36}
$$

证据下界记作: 

$$
\begin{eqnarray}
L(q) &=& E_{q}\left[\log p(x,z)\right] - E_{q}\left[\log q(z)\right] \\
&=& \int q(z)\log \frac{p(x,z)}{q(z)}\mathrm{~d}z  \\
&=& - D\left( q(z) \| p(x,z) \right)
\tag{20.37}
\end{eqnarray}
$$

<font color=blue>最小化KL散度（20.35）等价于求解证据下界的最大化问题</font>（目标是求$~q(z)~$使KL散度最小，此时$~\log p(x)~$是常量）。

**VI的另一种角度理解：**  

> 目标是通过证据$~\log p(x)~$的最大化，估计联合概率分布$~p(x,z)~$。因为含有隐变量$~z~$，直接对证据进行最大化困难，转而根据式（20.36）对证据下界进行最大化。

变分分布$~q(z)~$要求是容易处理的形式，通常假设$~q(z)~$对$~z~$的所有分量都是相互独立的，即满足  

$$
q(z) = q(z_{1})q(z_{2}) \cdots q(z_{n}) \tag{20.38}
$$

这时的变分分布称为<font color=blue>平均场（mean field）</font>。

&nbsp;

VI的几个步骤:  

（1）定义变分分布$q(z)$;  

（2）推导证据下界表达式$ELBO$;  

（3）对ELBO进行优化，如坐标上升法，得到最优分布$q^{*}(z)$，作为后验$~p(z|x)~$的近似。

### 20.4.1 变分EM算法

假设模型是联合概率分布$~p(x,z|\theta)~$，$~x~$是观测变量，$~z~$是隐变量，$~\theta~$是参数。  

目标是通过观测数据的概率（证据）$~\log p(x|\theta)~$的最大化，估计模型的参数$~\theta~$。使用VI，导入平均场 $q(z) = \prod_{i=1}^{n}q(z_{i})$，定义证据下界  

$$
L(q,\theta) = E_{q}\left[ \log p(x,z|\theta) \right] - E_{q}\left[ \log q(z) \right] \tag{20.39}
$$

通过迭代，分别以$~q~$和$~\theta~$为变量对证据下界进行最大化，就得到变分EM算法。

#### 算法20.3（变分EM算法）

循环执行一些 E 步和 M 步，直到收敛。  

（1）E步： 固定$~\theta~$，求$L(q,\theta)$对$~q~$的最大化；

（2）M步： 固定$~q~$，求$L(q,\theta)$对$~\theta~$的最大化；

给出模型参数$\theta$的估计值。

#### EM vs. 变分EM

EM的$Q$函数：$Q\left(\theta, \theta^{(i)}\right) = \sum_{Z}\log P(Y,Z|\theta)~P(Z|Y,\theta^{(i)})$

1. EM算法假设$q(z) = p(z|x)$ 且$p(z|x)$ 容易计算；
  
  
2. 变分EM算法则考虑一般情况使用容易计算的平均场$q(z) = \prod_{i=1}^{n}q(z_{i})$;


3. 当模型复杂时，EM算法未必可用，但变分EM算法仍然可以使用。